In [ ]:
!pip install accelerate transformers peft bitsandbytes datasets evaluate bert-score rouge-score


In [ ]:
from datasets import load_dataset, Dataset
import torch

In [ ]:
!pip install trl

In [ ]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig,DataCollatorForLanguageModeling
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model,PeftModel
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch
import os

In [ ]:
import pandas as pd

In [ ]:
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train")


data_df = pd.DataFrame(dataset)


In [ ]:
data_df = data_df.drop(columns=["id"])
data_df

In [ ]:
data_df.shape

In [ ]:
data_df["prompt"] = data_df.apply(
    lambda x: "Summarize the following article:\n" + x["article"] + "\n\nSummary:\n" + x["highlights"],
    axis=1
)


In [ ]:
data_df.head(5)

In [ ]:
class SFTFineTuner:
  def __init__(self,model_name,data_df,output_dir):
    """initalization of class parameter"""
    print("Parameters Intializaed")
    self.model_name=model_name
    self.data_df=data_df
    self.output_dir=output_dir
    self.tokenizer=None
    self.model=None
    self.tokenized_data=None


  def load_tokenizer(self):
    "defining tokenizer of the model"
    self.tokenizer=AutoTokenizer.from_pretrained(self.model_name,trust_remote_code=True)
    self.tokenizer.pad_token=self.tokenizer.eos_token
    self.tokenizer.padding_side = "left"  




  def load_model(self):
    "defining the model"

    ##Quantization
    bnb_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    self.model=AutoModelForCausalLM.from_pretrained(
        self.model_name,
        device_map={"":0},
        trust_remote_code=True,
        quantization_config=bnb_config
    )

    # Prepare the model for efficient k-bit (4-bit) training (QLoRA)
    self.model=prepare_model_for_kbit_training(self.model)



  def apply_lora(self):
    "applying lora to the model"
    config=LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj","v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    #applied lora on quatized model
    self.model=get_peft_model(self.model,config)
    self.model.print_trainable_parameters()

  def tokenize_dataset(self):
    "performing tokenization on the data"
    dataset = Dataset.from_pandas(self.data_df)

    # Tokenize the data
    def tokenize(sample):
        return self.tokenizer(sample["prompt"], padding=True, truncation=True, max_length=512)

    self.tokenized_data = dataset.map(tokenize, batched=True, desc="Tokenizing data", remove_columns=dataset.column_names)



  def train_model(self, epochs: int = 1, batch_size: int = 8, learning_rate: float = 2e-4, max_steps: int = 700):
      args = SFTConfig(
          output_dir=self.output_dir,               # Directory to save model checkpoints
          num_train_epochs=epochs,                  # Number of training epochs
          per_device_train_batch_size=batch_size,   # Batch size per GPU
          gradient_accumulation_steps=2,            # Accumulate gradients for larger effective batch
          gradient_checkpointing=True,              # Trade compute for memory savings
          optim="adamw_torch_fused",                # Use fused AdamW for efficiency
          learning_rate=learning_rate,              # Learning rate (QLoRA paper)
          max_grad_norm=0.3,                        # Gradient clipping threshold
          lr_scheduler_type="cosine_with_restarts", # Faster convergence
          warmup_ratio=0.1,              # Keep learning rate constant after warmup
          logging_steps=50,                        # Log metrics every N steps
          save_strategy="epoch",                    # Save checkpoint every epoch
          bf16=True,                                # Use bfloat16 precision
          push_to_hub=False,                        # Don't push to HuggingFace Hub
          report_to="none",                         # Disable external logging
          max_steps=max_steps,
          dataloader_num_workers=2,# Maximum number of training steps
      )
    
    # Remove the 'dataset_text_field' argument which is not accepted by SFTTrainer
      trainer = SFTTrainer(
          model=self.model,
          train_dataset=self.tokenized_data,
          args=args,
          data_collator=DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
      )

      trainer.train()

  
   

  def save_model(self):
    """
    this function will save the model
    """
    self.model.save_pretrained(self.output_dir)
    self.tokenizer.save_pretrained(self.output_dir)
    print(f"Adapter saved to {self.output_dir}")
    
    
      
  def run(self):
    """
    this function will run the whole process
    """
    print("starting finetunine process")
    self.load_tokenizer()
    print("tokenizer loaded")

    self.load_model()
    print("model loaded")

    self.apply_lora()
    print("lora applied")

    self.tokenize_dataset()
    print("dataset loaded and tokenized")

    self.train_model()
    print("model trained")

    self.save_model()
    print("model saved")


In [ ]:
model_name="mistralai/Mistral-7B-Instruct-v0.1"
output_dir="mistreal-7b-finetuned"


In [ ]:
fine_tuner=SFTFineTuner(model_name,data_df,output_dir)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
fine_tuner.run()

In [ ]:
def load_finetuned_model(model_name, adapter_path, device):
    # Load base model with same quantization config as training
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    
    base_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )
    
    # Attach adapter
    model = PeftModel.from_pretrained(base_model, adapter_path)
    tokenizer = AutoTokenizer.from_pretrained(adapter_path)
    
    return model, tokenizer


In [ ]:
def evaluate_article(article_text, tokenizer, model, device="cuda",
                     input_max_length=512, output_max_length=256):
    # Create prompt matching your training format
    prompt = f"""Summarize the following article:
{article_text}
Summary:"""
    
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=input_max_length,
        padding="max_length"
    ).to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=output_max_length,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract text after "Summary:" 
    summary = full_text.split("Summary:")[-1].strip()
    return summary

In [ ]:

# Usage -------------------------------------------------
import time

device = "cuda" if torch.cuda.is_available() else "cpu"


# Load your saved adapter (assuming output_dir is where you saved)
try:
    model, tokenizer = load_finetuned_model(
        model_name="mistralai/Mistral-7B-Instruct-v0.1",  # Original model name used in training
        adapter_path=output_dir,  # self.output_dir from SFTFineTuner
        device=device
    )
    model.eval()
    


    article_text = """
Governments worldwide are ramping up their climate action efforts ahead of the United Nations Climate
Summit scheduled for September 2025. With rising global temperatures and frequent extreme weather events, policymakers
are under increasing pressure to introduce stricter regulations and commit to bolder carbon reduction targets.In Europe,
the European Union (EU) announced plans to implement more aggressive emission reduction measures. The EU Commission proposed
a 60% reduction in greenhouse gas emissions by 2035, surpassing the previously set target of 55%. Additionally, they aim to expand
renewable energy projects, including solar and wind farms, across member states.Meanwhile, in the United States, the Biden administration
introduced new federal guidelines aimed at accelerating the transition to electric vehicles (EVs). The Environmental Protection Agency (EPA)
proposed regulations that would require automakers to ensure that 67% of new vehicle sales by 2032 are electric. This move is part of the
administration's broader goal of achieving net-zero emissions by 2050.
"""  # Your article text
    start_time=time.time()

    generated_summary = evaluate_article(
        article_text,
        tokenizer=tokenizer,
        model=model,
        device=device
    )
    
    print(f"Generated article summary in {time.time()-start_time:.2f}s")
    print("Generated Summary:\n", generated_summary)
    

In [ ]:


    # Load the CNN/DailyMail dataset again to get reference summaries
    eval_dataset = load_dataset("cnn_dailymail", "3.0.0", split="test")
    eval_df = pd.DataFrame(eval_dataset)

    # Select a few examples for evaluation (e.g., first 5)
    num_eval_examples = 5
    evaluation_articles = eval_df['article'].tolist()[:num_eval_examples]
    reference_summaries = eval_df['highlights'].tolist()[:num_eval_examples]
    

    generated_summaries =[]
    for article_text in evaluation_articles:
        generated_summary = evaluate_article(
            article_text,
            tokenizer=tokenizer,
            model=model,
            device=device
        )
        generated_summaries.append(generated_summary)

    # Load ROUGE metric
    rouge = evaluate.load("rouge")

    # Compute ROUGE scores
    rouge_results = rouge.compute(
        predictions=generated_summaries,
        references=reference_summaries,
        rouge_types=["rouge1", "rouge2", "rougeL"]
    )

    print("\nROUGE Scores:")
    print(f"ROUGE-1: {rouge_results['rouge1']}")
    print(f"ROUGE-2: {rouge_results['rouge2']}")
    print(f"ROUGE-L: {rouge_results['rougeL']}")

    # Load BERTScore metric
    bertscore = evaluate.load("bertscore")

    # Compute BERTScore
    bertscore_results = bertscore.compute(
        predictions=generated_summaries,
        references=reference_summaries,
        lang="en",
        model_type="microsoft/deberta-xlarge-mnli"   # You can choose a different model
    )

    print("\nBERTScore (F1):")
    print(f"Average: {np.mean(bertscore_results['f1']):.4f}")
    print(f"Min: {np.min(bertscore_results['f1']):.4f}")
    print(f"Max: {np.max(bertscore_results['f1']):.4f}")

except Exception as e:
    print(f"An error occurred during evaluation: {e}")
    print("Make sure you have trained and saved the model adapter to the specified output directory.")
    print("Also, ensure you have the necessary libraries installed (evaluate, bert_score).")